<a href="https://colab.research.google.com/github/arminwitte/llsi/blob/notebooks/example_diss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llsi

In [2]:
!wget https://raw.githubusercontent.com/arminwitte/llsi/80150bc50595bbd6ba2e06b351b0c439b023c867/data/heated_wire_data.npy

--2024-09-07 21:54:57--  https://raw.githubusercontent.com/arminwitte/llsi/80150bc50595bbd6ba2e06b351b0c439b023c867/data/heated_wire_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7152728 (6.8M) [application/octet-stream]
Saving to: ‘heated_wire_data.npy.9’

heated_wire_data.np 100%[===================>]   6.82M  6.28MB/s    in 1.1s    

2024-09-07 21:54:58 (6.28 MB/s) - ‘heated_wire_data.npy.9’ saved [7152728/7152728]



In [3]:
# %cd ..

In [4]:
!pip install -e .

Obtaining file:///home/jovyan
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llsi (pyproject.toml) ... done
  Created wheel for llsi: filename=llsi-0.3.1-py3-none-any.whl size=3831 sha256=fecfd8005002e3f080c388be5a9e3f0f31addad2452556980cb7d4c60486a792
  Stored in directory: /tmp/pip-ephem-wheel-cache-kh47_39j/wheels/bc/01/de/a88c3880a4c27c802b27f000f770a2fe391bdc11e738d133f7
Successfully built llsi
  Attempting uninstall: llsi
    Found existing installation: llsi 0.3.1
    Uninstalling llsi-0.3.1:
      Successfully uninstalled llsi-0.3.1


In [5]:
import matplotlib.pyplot as plt
import numpy as np

import llsi

In [6]:
d = np.load("notebooks/heated_wire_data.npy")

In [7]:
t = d[:, 0]
Re = d[:, 1]
Nu = d[:, 2]

In [8]:
data = llsi.SysIdData(t=t, Re=Re, Nu=Nu)
data.equidistant(305002)
print(data.time().shape)
data.center()
data.downsample(18)
data.lowpass(1, 10)
data, test_set = data.split(0.8)
data.crop(start=1000)
data.center()
print(1 / data.Ts)

(305002,)
Splitting at 13556
239.9995820843808


In [ ]:
with llsi.Figure() as fig:
    mod1 = llsi.sysid(
        data, "Nu", "Re", (0, 100, 0), method="arx", settings={"lambda": 1e2}
    )
    mod1 = llsi.sysid(data, "Nu", "Re", 2, method="pem", settings={"init": "n4sid"})
    print("================")
    mod2 = llsi.sysid(
        data, "Nu", "Re", (3, 3, 0), method="pem", settings={"init": "arx"}
    )
    fig.plot([mod1, mod2], "impulse")
    fig.plot(
        {"mod": [mod1, mod2], "data": test_set, "y_name": "Nu", "u_name": "Re"},
        "compare",
    )

M shape: (15, 1)
 0.0499612
 0.0499612
 0.0499612
 0.0499613
 0.0499612
 0.0499596
 0.0499598
 0.0499612
 0.0499612
 0.0499615
 0.0499612
 0.0499612
    218512
    218512
    218512
    218512
    218512
    218512
    218512
    218512
    218512
    218512
    218512
    218512
 0.0532261
 0.0532261
 0.0532261
  0.053226
 0.0532261
 0.0532263
   0.05323
 0.0532261
 0.0532261
 0.0532263
 0.0532261
 0.0532261
 0.0439899
 0.0439898
 0.0439899
 0.0439898
 0.0439899
 0.0439891
 0.0439908
 0.0439898
 0.0439899
 0.0439901
 0.0439899
 0.0439899
 0.0332118
 0.0332118
 0.0332118
 0.0332118
 0.0332118
 0.0332112
 0.0332126
 0.0332118
 0.0332118
  0.033212
 0.0332118
 0.0332118
 0.0269379
 0.0269379
 0.0269379
 0.0269378
 0.0269379
 0.0269371
 0.0269383
 0.0269379
 0.0269379
  0.026938
 0.0269379
 0.0269379
 0.0206983
 0.0206983
 0.0206983
 0.0206983
 0.0206983
 0.0206974
 0.0206982
 0.0206983
 0.0206983
 0.0206984
 0.0206983
 0.0206983
 0.0133749
 0.0133749
 0.0133749
 0.0133749
 0.0133749
 0.0

In [ ]:
if False:
    import scipy

    t, y = scipy.signal.dimpulse(mod1.to_ss())

    fig, ax = plt.subplots()
    plt.plot(t, np.squeeze(y))
    plt.grid()
    plt.xlabel("n [samples]")
    plt.ylabel("Amplitude")

    t, y = scipy.signal.impulse(mod1.to_ss(continuous=True))

    # fig, ax = plt.subplots()
    plt.plot(t[1:], np.squeeze(y)[1:])
    plt.grid()
    plt.xlabel("n [samples]")
    plt.ylabel("Amplitude")

    t, y = scipy.signal.impulse(mod1.to_ss(continuous=True, method="euler"))

    # fig, ax = plt.subplots()
    plt.plot(t[1:], np.squeeze(y)[1:])
    plt.grid()
    plt.xlabel("n [samples]")
    plt.ylabel("Amplitude")

    t, y = scipy.signal.dimpulse(mod1.to_tf())
    plt.plot(t[1:], np.squeeze(y)[1:])
    plt.grid()
    plt.xlabel("n [samples]")
    plt.ylabel("Amplitude")

    t, y = scipy.signal.dimpulse(mod1.to_zpk())
    plt.plot(t[1:], np.squeeze(y)[1:])
    plt.grid()
    plt.xlabel("n [samples]")
    plt.ylabel("Amplitude")

    plt.legend(["discrete", "cont", "euler", "tf", "zpk"])

In [ ]:
fig, ax = plt.subplots()
ti1, i1 = mod1.impulse_response()
ti2, i2 = mod2.impulse_response(300)
plt.plot(ti1[50:], i1[50:])
plt.plot(ti2[50:], i2[50:], "r")
plt.grid(True)

print(i1[:5])
# print(i2[:5])